In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU was detected. CNNs can be very slow without a GPU.
Go to Runtime > Change runtime and select a GPU hardware accelerator.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
#신경망 구성

In [ ]:
#데이터 셋 나누기
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
#standardization
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7 #standardization 수행
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]), #커널 7x7, 필터 64개
    keras.layers.MaxPooling2D(pool_size=2), #크기 1/2로 줄어듦
    DefaultConv2D(filters=128), #나머지는 default로
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(), #중간에 flatten
    keras.layers.Dense(units=128, activation='relu'), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),#최종적으로 10개 분류하는거니까 units을 10으로 하고, 분류 하는 목적이니까 softmax 사용
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=[X_valid, y_valid])
score = model.evaluate(X_test, y_test)
X_new = X_test[:10] # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 804s 467ms/step - loss: 0.7201 - accuracy: 0.7491 - val_loss: 0.3927 - val_accuracy: 0.8548
Epoch 2/10
1719/1719 [==============================] - 799s 465ms/step - loss: 0.4183 - accuracy: 0.8596 - val_loss: 0.3428 - val_accuracy: 0.8796
Epoch 3/10
1719/1719 [==============================] - 801s 466ms/step - loss: 0.3608 - accuracy: 0.8791 - val_loss: 0.3243 - val_accuracy: 0.8848
Epoch 4/10
1719/1719 [==============================] - 804s 468ms/step - loss: 0.3285 - accuracy: 0.8912 - val_loss: 0.2936 - val_accuracy: 0.8968
Epoch 5/10
1719/1719 [==============================] - 807s 470ms/step - loss: 0.3103 - accuracy: 0.8956 - val_loss: 0.2887 - val_accuracy: 0.8994
Epoch 6/10
1719/1719 [==============================] - 805s 468ms/step - loss: 0.2891 - accuracy: 0.9023 - val_loss: 0.2838 - val_accuracy: 0.9014
Epoch 7/10
1719/1719 [==============================] - 804s 467ms/step - loss: 0.2738 - accuracy: 0.9079 - val_